<a href="https://colab.research.google.com/github/seeouy/edgetpu_model_converter/blob/master/keras_to_edgetpu_model_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

既存のtensorflowをアンインストールして、tensorflow2.0をインストールしなおします。

※ランタイムのタイプを「GPU」に設定してください。

In [1]:
! pip uninstall -y tensorflow
! pip uninstall -y tensorflow-gpu
# GPU搭載の環境
! pip install tensorflow-gpu==2.0-beta1

Uninstalling tensorflow-gpu-2.0.0b1:
  Successfully uninstalled tensorflow-gpu-2.0.0b1
  Using cached https://files.pythonhosted.org/packages/2b/53/e18c5e7a2263d3581a979645a185804782e59b8e13f42b9c3c3cfb5bb503/tensorflow_gpu-2.0.0b1-cp36-cp36m-manylinux1_x86_64.whl


kerasモデル(.h5)の作成。

すでにトレーニング済みのモデルがあれば、このステップは不要。


In [2]:
import numpy as np
import tensorflow as tf
print(f'TensorFlow\'s version: {tf.__version__}')
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.datasets  import fashion_mnist
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train = x_train / 255.0
x_test = x_test / 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# train_images = train_images[:, :, :, np.newaxis].astype(np.float32)
# test_images = test_images[:, :, :, np.newaxis].astype(np.float32)

# train_images = train_images[:, :, :, np.newaxis].astype(np.float32)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# save model
model.save("model_fashion-mnist_cnn.h5")

TensorFlow's version: 2.0.0-beta1
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


W0721 15:16:21.991582 139858891495296 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 10s 160us/sample - loss: 2.2876 - accuracy: 0.1348 - val_loss: 2.2493 - val_accuracy: 0.3060
Epoch 2/12
60000/60000 [==============================] - 8s 135us/sample - loss: 2.2241 - accuracy: 0.2335 - val_loss: 2.1721 - val_accuracy: 0.3973
Epoch 3/12
60000/60000 [==============================] - 8s 136us/sample - loss: 2.1403 - accuracy: 0.3209 - val_loss: 2.0685 - val_accuracy: 0.4469
Epoch 4/12
60000/60000 [==============================] - 8s 135us/sample - loss: 2.0289 - accuracy: 0.3851 - val_loss: 1.9309 - val_accuracy: 0.5345
Epoch 5/12
60000/60000 [==============================] - 8s 135us/sample - loss: 1.8907 - accuracy: 0.4350 - val_loss: 1.7624 - val_accuracy: 0.5485
Epoch 6/12
60000/60000 [==============================] - 8s 135us/sample - loss: 1.7381 - accuracy: 0.4696 - val_loss: 1.5838 - val_accuracy: 0.5543
Epoch 7/12
60000/60000 [=========================

keras(tensorflow)モデルをtensorflow-lite形式のモデルに変換。

変換対象のモデル名を「model_name」に指定します。

In [0]:
model_name = "model_fashion-mnist_cnn.h5"

# 量子化はデータのキャリブレーション用の「representative data set」が必要
# ここでは、x_trainの一部(1000）を用いる
def representative_dataset_gen():
    for i in range(100):
        yield [x_train[i: i + 1]]
        
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(model_name)
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
with open(model_name + '.tflite', 'wb') as o_:
    o_.write(tflite_model)

edgetpuコンパイラのインストール。

[インストール方法の参照先](https://coral.withgoogle.com/docs/edgetpu/compiler/)

In [4]:
%%bash

echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 6A030B21BA07F4FB

sudo apt update

sudo apt install edgetpu

deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Executing: /tmp/apt-key-gpghome.8WWNmJzPwH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 6A030B21BA07F4FB
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidi

gpg: key 6A030B21BA07F4FB: "Google Cloud Packages Automatic Signing Key <gc-team@google.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1






tfliteモデルをedtetpuモデルに変換します。

※ファイル名を正しく指定しましょう(model_name + ".tflite")。

　出力ファイルは、model_name + "_edgetpu.tflite"

In [5]:
!edgetpu_compiler --show_operations 'model_fashion-mnist_cnn.h5.tflite'

Edge TPU Compiler version 1.0.249710469
INFO: Initialized TensorFlow Lite runtime.

Model compiled successfully in 99 ms.

Input model: model_fashion-mnist_cnn.h5.tflite
Input size: 1.15MiB
Output model: model_fashion-mnist_cnn.h5_edgetpu.tflite
Output size: 1.21MiB
On-chip memory available for caching model parameters: 7.83MiB
On-chip memory used for caching model parameters: 30.00KiB
Off-chip memory used for streaming uncached model parameters: 1.13MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 8
Operation log: model_fashion-mnist_cnn.h5_edgetpu.log

Operator                       Count      Status

MAX_POOL_2D                    1          Mapped to Edge TPU
QUANTIZE                       2          Mapped to Edge TPU
CONV_2D                        1          Mapped to Edge TPU
DEPTHWISE_CONV_2D              1          Mapped to Edge TPU
SOFTMAX                        1          Mapped to Edge TPU
FULLY_CONNECTED                2          Mapped to Edge TPU


モデルをローカルに保存。

In [0]:
from google.colab import files
files.download( "./model_fashion-mnist_cnn.h5.tflite" ) 
files.download( "./model_fashion-mnist_cnn.h5_edgetpu.tflite" ) 